In [17]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
from sklearn import preprocessing
 
import jsonx
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report,confusion_matrix 

In [3]:
def load_dataset(json_file):
    json_f=open(json_file).read() 
    json_info=json.loads(json_f)
    json_data=pd.json_normalize(json_info) # already normalization
    #print('json_data shape',json_data.shape)
    #print('json_Data columns',json_data.columns)
    return json_data
 
def get_fpr(y,y_pred):   
    cm = confusion_matrix(y, y_pred)
    tn=cm[0, 0]
    fp=cm[0, 1]
    fn=cm[1, 0]
    tp=cm[1, 1]
    tpr=tp/(tp+fn)
    tnr=tn/(tn+fp)
    f1_score=2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn))))
    #tnr=format(tnr,'.4f')
    #tpr=format(tpr,'.2f')
    return tnr,tpr,f1_score

# combine the features to train models, recall performance

In [8]:
#english data
 
en_feature="../data/engphish/chspec_combine_enphish_full.json" #15111
wst_feature="../data/wstphish/chspec_combine_western_full.json"#11204
chi_feature="../data/chiphish/chspec_combine_chphish_full.json"#1620

en_data=load_dataset(en_feature)
print('en_data is',en_data.shape)
#en_data['label'].value_counts()
wst_data=load_dataset(wst_feature)
print('wst_data is',wst_data.shape)
chi_data=load_dataset(chi_feature)
print('chi_data is',chi_data.shape)


en_data is (15111, 69)
wst_data is (11204, 69)
chi_data is (1620, 69)


In [9]:

data_en=en_data.iloc[:,:66]
data_en=data_en.astype(float)
print('english dataset is')
#data_en.columns
#print('the first data of data_en',data_en.iloc[0])

print('chi dataset is')
data_chi=chi_data.iloc[:,:66]
data_chi=data_chi.astype(float)
#data_chi.columns
#print('the first data of data_chi',data_chi.iloc[0])

data_wst=wst_data.iloc[:,:66]
data_wst=data_wst.astype(float)
print('wst dataset is')
#data_wst.columns
#print('the first data of data_wst',data_wst.iloc[0])

english dataset is
chi dataset is
wst dataset is


In [12]:
train_en,test_en=train_test_split(data_en,test_size=0.2,shuffle=True)
train_chi,test_chi=train_test_split(data_chi,test_size=0.2,shuffle=True)
train_wst,test_wst=train_test_split(data_wst,test_size=0.2,shuffle=True)
# print(type(train_en))
# print(train_en.shape)
# print('en columns is',train_en.columns)

train_en_chi_wst = pd.concat([train_en, train_chi,train_wst])

print(train_en_chi_wst.shape)
print('en chi columns is',train_en_chi_wst.columns)

(22347, 66)
en chi columns is Index(['URL_length', 'URL_IP', 'URL_redirect', 'URL_shortener',
       'URL_subdomains', 'URL_at', 'URL_dash', 'URL_numberofCommonTerms',
       'URL_checkNumerical', 'URL_checkPathExtend', 'URL_checkPunycode',
       'URL_checkSensitiveWord', 'URL_checkTLDinPath', 'URL_checkTLDinSub',
       'URL_totalWordUrl', 'URL_shortestWordUrl', 'URL_shortestWordHost',
       'URL_shortestWordPath', 'URL_longestWordUrl', 'URL_longestWordHost',
       'URL_longestWordPath', 'URL_averageWordUrl', 'URL_averageWordHost',
       'URL_averageWordPath', 'URL_checkStatisticRe', 'REP_SearchEngine',
       'REP_checkGI', 'REP_pageRank', 'REP_DNS', 'REP_registrationLen',
       'REP_Age', 'REP_abnormal', 'REP_ports', 'REP_SSL', 'url_unicode',
       'tld_number', 'icp_applicant', 'icp_domain', 'icp_dregister',
       'HTML_Objects', 'HTML_metaScripts', 'HTML_FrequentDomain',
       'HTML_Commonpage', 'HTML_CommonPageRatioinFooter', 'HTML_SFH',
       'HTML_popUp', 'HTML_RightCl

# Universal RF-based PWD using Fc, refer to Table VI

In [19]:
# run all the tests 10 times to get the mean results
def full_metri(model,train_x,train_y,test_x,test_y):
    model.fit(train_x,train_y)
    pred=model.predict(test_x)
    tnr,tpr,f1_score=get_fpr(test_y,pred)
    print('tnr is %.2f, recall is %.2f,f1 is %.2f'%(tnr,tpr,f1_score))
    return model,tnr,tpr,f1_score

def pre_metri(model,test_x,test_y):
    #model.fit(train_x,train_y)
    pred=model.predict(test_x)
    tnr,tpr,f1_score=get_fpr(test_y,pred)
    print('tnr is %.2f, recall is %.2f,f1 is %.2f'%(tnr,tpr,f1_score))
    return tnr,tpr,f1_score

train_en,test_en=train_test_split(data_en,test_size=0.2,shuffle=True)
train_chi,test_chi=train_test_split(data_chi,test_size=0.2,shuffle=True)
train_wst,test_wst=train_test_split(data_wst,test_size=0.2,shuffle=True)
train_all = pd.concat([train_en, train_chi,train_wst])
 

train_all_x=train_all.iloc[:,0:65]
train_all_y=train_all.iloc[:,65] 

# test sets
test_en_x=test_en.iloc[:,0:65] 
test_en_y=test_en.iloc[:,65] 

test_chi_x=test_chi.iloc[:,0:65]
test_chi_y=test_chi.iloc[:,65]

test_wst_x=test_wst.iloc[:,0:65]
test_wst_y=test_wst.iloc[:,65]

scaler_entire_all=preprocessing.MinMaxScaler()
scaler_entire_all.fit(train_all_x)

train_x_all=scaler_entire_all.transform(train_all_x)
test_x_en=scaler_entire_all.transform(test_en_x)
test_x_chi=scaler_entire_all.transform(test_chi_x)
test_x_wst=scaler_entire_all.transform(test_wst_x)


model_rf_all=RandomForestClassifier(criterion='gini', max_depth=30, min_samples_split=5, n_estimators=200,random_state=8)#9
# print('combine_rf_whole:')
model_rf_com,rf_tnr_com_en,rf_recall_com_en,rf_f1_com_en=full_metri(model_rf_all,train_x_all,train_all_y,test_x_en,test_en_y)#using normalized data to train, so all the model_rf4_com data must normalized
#model_rf_com=joblib.load('./all_model_spec/model_rf_com_full_'+str(i)+'.pkl')
# rf_tnr_com_en,rf_tpr_com_en,rf_f1_com_en=pre_metri(model_rf_com,test_x_en,test_en_y)
rf_tnr_com_chi,rf_tpr_com_chi,rf_f1_com_chi=pre_metri(model_rf_com,test_x_chi,test_chi_y)
rf_tnr_com_wst,rf_tpr_com_wst,rf_f1_com_wst=pre_metri(model_rf_com,test_x_wst,test_wst_y)

# if format(rf_tpr_com_chi,2)==0.89 and rf_tnr_com_wst==0.95 and rf_f1_com_chi==0.94
    
    

MinMaxScaler()

tnr is 1.00, recall is 0.97,f1 is 0.99
tnr is 0.99, recall is 0.86,f1 is 0.92
tnr is 0.95, recall is 0.99,f1 is 0.98
